In [1]:
# Download
!git clone https://github.com/nightingal3/Fig-QA
%cd Fig-QA/

Cloning into 'Fig-QA'...
remote: Enumerating objects: 639, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 639 (delta 130), reused 139 (delta 88), pack-reused 431
Receiving objects: 100% (639/639), 2.81 MiB | 11.21 MiB/s, done.
Resolving deltas: 100% (353/353), done.
/kaggle/working/Fig-QA


In [2]:
# install
%pip install transformers[torch]
%pip install accelerate -U
%pip install deepspeed

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.2 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.11.1-py3-none-any.whl size=1047120 sha256=2d3907bd7ea85a04b5381af7b10249d64ef3374af93a197fe9f2f3375c46a55c
  Stored in directory: /root/.cache/pip/wheels/fc/a5/a7/bc3e8d6eea7ae73f7f3ea95f4caaba73a2853d70d5e8a9301a
Successfully built deepspeed
Note: you may need to restart the kernel to use updated packages.


# Try Running Their Script
python3 src/models/train_lm_models.py {gpt2,gpt-neo-sm,gpt-neo-lg} \
[--dont_train] \
[--dont_eval] \
[--train_path=TRAIN_PATH] \
[--eval_path=EVAL_PATH] \
[--seed=SEED] \
[--cuda] \
[--num_epochs=NUM_EPOCHS] \
[--learning_rate=LR] \
[--middle_phrase=SUFFIX_PROMPT] \
[--prefix=N] \
[--contrastive] \
[--contrast_lambd=a] \
[--log_history] \
[--deepspeed] \
[----out_path=PATH] \
[----early_stopping]

In [3]:
#!python3 src/models/train_lm_models.py gpt2 --cuda

In [4]:
#!python3 src/models/train_lm_models.py gpt-neo-sm --cuda

# Modifying code

### Changes:
- modify model_init to use correct loader for model
- add model string to main function

In [5]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, GPTNeoForCausalLM, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def model_init(model_string, cuda, output_attentions=False, fast=False):
    if model_string.startswith("gpt2"):
        if fast:
            tokenizer = AutoTokenizer.from_pretrained(model_string)
            model = GPT2LMHeadModel.from_pretrained(model_string)
        else:
            tokenizer = GPT2Tokenizer.from_pretrained(model_string)
            model = GPT2LMHeadModel.from_pretrained(model_string)
    elif model_string.startswith("EleutherAI/gpt-neo"):
        tokenizer = GPT2Tokenizer.from_pretrained(model_string, output_attentions=output_attentions)
        model = GPTNeoForCausalLM.from_pretrained(model_string, output_attentions=output_attentions)
    elif "t5" in model_string:
      tokenizer = AutoTokenizer.from_pretrained(model_string)
      model = AutoModelForSeq2SeqLM.from_pretrained(model_string)
    else:
        tokenizer = OpenAIGPTTokenizer.from_pretrained(model_string)
        model = OpenAIGPTLMHeadModel.from_pretrained(model_string)
    model.eval()
    if cuda:
        model.to('cuda')
    return model, tokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
import argparse
import logging
from typing import Optional
from glob import glob
from pathlib import Path
import os, sys
import torch
import numpy as np
import pandas as pd
import pickle

import transformers
from transformers import (
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    LineByLineWithRefDataset,
    PreTrainedTokenizer,
    TextDataset,
    Trainer,
    TrainingArguments,
    set_seed,
    GPT2LMHeadModel,
    GPTNeoForCausalLM,
    EarlyStoppingCallback
)
from torch.utils.data import ConcatDataset
import pdb

# Add path for those local py modules
sys.path.append('src/models/')
from gpt_score import evaluate_model

logger = logging.getLogger(__name__)

def main(model_name: str, prompt: str, train_path: str, eval_path: str, contrastive_train: bool, contrastive_train_lambd: float, num_epochs: int, seed: int, lr: int, use_cuda: bool, dont_train: bool, dont_eval: bool, out_path: str, cache_dir: str = "./lm_train_cache/", prefix_prompt: int = 0, batch_size: int = 2, log_history: bool = False, deepspeed: bool = False, early_stopping: bool = False, patience_num : int = 3) -> None:
    # Set up models, random seed, and logging
    model_names = {
        "gpt2": "gpt2",
        "gpt-neo-sm": "EleutherAI/gpt-neo-1.3B",
        "gpt-neo-lg": "EleutherAI/gpt-neo-2.7B",
        # new update
        "gpt-neo-sssm": "EleutherAI/gpt-neo-125m",
        "flan-t5-base":"google/flan-t5-base",
        "flan-t5-small" : "google/flan-t5-small"
    }
    model_id = model_names[model_name]
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    transformers.utils.logging.set_verbosity_info()
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()
    logger.info("Training/evaluation parameters %s", {"model": model_name, "train path": train_path, "num epochs": num_epochs, "seed": seed, "cuda": use_cuda, "cache dir": cache_dir, "deepspeed": deepspeed, "early stopping": early_stopping})


    if deepspeed and not use_cuda:
        logger.info("You must have GPUs to use deepspeed. Turning cuda flag on...")
        use_cuda = True

    model, tokenizer = model_init(model_id, use_cuda, fast=True)
    tokenizer.pad_token = tokenizer.eos_token
    #model.resize_token_embeddings(len(tokenizer))
    set_seed(seed)

    # load datasets and initialize trainer
    train_dataset = (
        get_dataset(train_path, tokenizer=tokenizer, cache_dir=cache_dir)
    )
    eval_dataset = (
        get_dataset(eval_path, tokenizer=tokenizer, cache_dir=cache_dir)
    )

    eval_df = pd.read_csv("./data/filtered/dev.csv")
    eval_df["label"] = eval_df["labels"]
    test_df = pd.read_csv("./data/filtered/dev.csv")
    test_df["label"] = test_df["labels"]

    data_collator = DataCollatorForLanguageModeling(
                tokenizer=tokenizer, mlm=False
            )
    no_cuda = not use_cuda

    default_arguments = {
        "output_dir": f"./lm_train_outputs/{model_name}_{seed}/",
        "do_train": True,
        "prediction_loss_only": False,
        "num_train_epochs": num_epochs,
        "seed": seed,
        "learning_rate": lr,
        "per_device_train_batch_size": batch_size,
        "per_device_eval_batch_size": batch_size,
        "no_cuda": no_cuda,
        "report_to":"none",
        "lr_scheduler_type":"cosine"
    }

    if deepspeed:
        default_arguments["deepspeed"] = "deepspeed_config.json"
    if not contrastive_train:
        default_arguments["per_device_train_batch_size"] = batch_size
        default_arguments["per_device_eval_batch_size"] = batch_size

    else:
        default_arguments["per_device_train_batch_size"] = 2

    if log_history:
        default_arguments["evaluation_strategy"] = "steps"
        default_arguments["eval_steps"] = 100
    if early_stopping:
        default_arguments["evaluation_strategy"] = "epoch"
        default_arguments["load_best_model_at_end"] = True
        default_arguments["metric_for_best_model"] = "eval_loss"
        default_arguments["save_strategy"] = "epoch"
        default_arguments["save_total_limit"] = 2  # Keeps only 2 recent checkpoints plus the best model

    training_args = transformers.TrainingArguments(**default_arguments)

    if early_stopping:
        trainer = Trainer(
            args=training_args,
            model=model,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            callbacks=[EarlyStoppingCallback(early_stopping_patience= patience_num)]
        )
    elif not contrastive_train:
        #tokenizer.pad_token = tokenizer.eos_token
        #dummy_init = make_dummy(model_id)
        trainer = Trainer(
            args=training_args,
            model=model,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            #model_init=dummy_init,
            compute_metrics=compute_metrics
        )
    else:
        trainer = ContrastiveTrainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        trainer.set_lambd(contrastive_train_lambd)

    # Train the model
    if not dont_train:
        logger.info("=== Training the model ===")
        trainer.train()
        trainer.save_model("./lm_train_cache/")
        if log_history:
            log_file = f"{model_name}_epochs_{num_epochs}_eval_loss.p"
            with open(log_file, "wb") as f:
                pickle.dump(trainer.state.log_history, f)

    # Evaluate the model
    results = {}
    if not dont_eval: #Note: for hyperparameter tuning we do it by loss on
        model.eval()
        logger.info("=== Evaluating the model ===")
        eval_output = trainer.evaluate()
        eval_loss = eval_output["eval_loss"]
        results["eval_loss"] = eval_loss

        acc_test, out_df_test, preds_test, labels_test = evaluate_model(model, tokenizer, test_df.to_dict(orient="records"), use_cuda=use_cuda, return_acc=True, middle_phrase=prompt, use_prefix=prefix_prompt)
        acc_dev, out_df_dev, preds_dev, labels_dev = evaluate_model(model, tokenizer, eval_df.to_dict(orient="records"), use_cuda=use_cuda, return_acc=True, middle_phrase=prompt, use_prefix=prefix_prompt)
        results["accuracy (test)"] = acc_test
        results["accuracy (dev)"] = acc_dev
        results["preds"] = preds_test
        results["labels"] = labels_test


    if out_path is not None:
        Path(out_path).mkdir(parents=True, exist_ok=True)
        with open(f"{out_path}/results_{model_name}.txt", "w") as writer:
            logger.info("=== Outputting results ===")
            for key in sorted(results.keys()):
                logger.info("  %s = %s", key, str(results[key]))
                writer.write("%s = %s\n" % (key, str(results[key])))

        out_df_test.to_csv(f"{out_path}/prob_{model_name}_{seed}.csv", index=False)

    return results

def training_setup(model, tokenizer, model_name, seed, lr, num_epochs, train_path, eval_path, contrastive_train=False, contrast_lambd=1, is_hyperparam_opt=False, cuda=True, deepspeed=False, batch_size=8) -> Trainer:
    # load datasets and initialize trainer
    train_dataset = (
        get_dataset(train_path, tokenizer=tokenizer)
    )
    eval_dataset = (
        get_dataset(eval_path, tokenizer=tokenizer)
    )

    data_collator = DataCollatorForLanguageModeling(
                tokenizer=tokenizer, mlm=False
            )
    set_seed(seed)

    default_train_args = {
        "output_dir": f"./lm_train_outputs/{model_name}_{seed}/",
        "do_train": True,
        "do_eval": False,
        "prediction_loss_only": True,
        "seed": seed,
        "num_train_epochs": num_epochs,
        "learning_rate": lr,
        "no_cuda": not cuda,
        "per_device_train_batch_size": batch_size,
        "per_device_eval_batch_size": batch_size,
        "report_to":"none"
    }

    if contrastive_train:
        default_train_args["per_device_train_batch_size"] = 2
        training_args = transformers.TrainingArguments(output_dir=f"./lm_train_outputs/{model_name}_{seed}/", do_train=True, do_eval=False,
        prediction_loss_only=True, num_train_epochs=num_epochs, seed=seed,learning_rate=lr, per_device_train_batch_size=2, report_to="none")
    elif is_hyperparam_opt:
        default_train_args["evaluation_strategy"] = "steps"
        default_train_args["eval_steps"] = 500
        default_train_args["disable_tqdm"] = True
    if deepspeed == True:
        default_train_args["deepspeed"] = "./deepspeed_config.json"

    training_args = transformers.TrainingArguments(**default_train_args)


    if is_hyperparam_opt:
        tokenizer.pad_token = tokenizer.eos_token
        dummy_init = make_dummy(model_name)
        trainer = Trainer(
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            model_init=dummy_init,
            compute_metrics=compute_metrics
        )
    elif contrastive_train:
        trainer = ContrastiveTrainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        trainer.set_lambd(contrast_lambd)
    else:
        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
    return trainer

# This is adapted from the huggingface LM training example here: https://github.com/huggingface/transformers/blob/master/examples/legacy/run_language_modeling.py
def get_dataset(
    train_data_file: str,
    tokenizer: PreTrainedTokenizer,
    line_by_line: bool = True,
    evaluate: bool = False,
    eval_data_file: str = None,
    cache_dir: Optional[str] = None,
):
    def _dataset(file_path, ref_path=None):
        if line_by_line:
            if ref_path is not None:
                if not args.whole_word_mask or not args.mlm:
                    raise ValueError("You need to set world whole masking and mlm to True for Chinese Whole Word Mask")
                return LineByLineWithRefDataset(
                    tokenizer=tokenizer,
                    file_path=file_path,
                    block_size=tokenizer.model_max_length,
                    ref_path=ref_path,
                )

            return LineByLineTextDataset(tokenizer=tokenizer, file_path=file_path, block_size=tokenizer.model_max_length)

    if evaluate:
        return _dataset(eval_data_file)
    else:
        return _dataset(train_data_file)

def make_dummy(model_id):
    def dummy_init():
        if model_id == "gpt2":
            return GPT2LMHeadModel.from_pretrained("gpt2", return_dict=True)
        elif "gpt-neo" in model_id:
            return GPTNeoForCausalLM.from_pretrained(model_id, return_dict=True)
    return dummy_init

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    acc = len(np.where(predictions == labels)[0])/len(labels)
    return {"acc": acc}

class ContrastiveTrainer(Trainer):
    def set_lambd(self, lambd):
        self.lambd = lambd

    def compute_loss(self, model, inputs, return_outputs=False):
        # Assumes batch size of 2!
        if inputs["labels"].shape[0] % 2 != 0:
            raise ValueError("Batch size must be a multiple of 2")

        correct_inputs = {"input_ids": torch.stack([row for i, row in enumerate(inputs["input_ids"]) if i % 2 == 0]),
        "attention_mask": torch.stack([row for i, row in enumerate(inputs["attention_mask"]) if i % 2 == 0]),
        "labels":  torch.stack([row for i, row in enumerate(inputs["labels"]) if i % 2 == 0])}
        wrong_inputs = {"input_ids": torch.stack([row for i, row in enumerate(inputs["input_ids"]) if i % 2 == 1]),
        "attention_mask": torch.stack([row for i, row in enumerate(inputs["attention_mask"]) if i % 2 == 1]),
        "labels":  torch.stack([row for i, row in enumerate(inputs["labels"]) if i % 2 == 1])}

        outputs = model(**inputs)

        correct_outputs = model(**correct_inputs)
        correct_loss = correct_outputs.get('loss')

        wrong_outputs = model(**wrong_inputs)
        wrong_loss = wrong_outputs.get("loss")

        # Good = when the loss for the correct item is much lower than loss for wrong item
        # loss should be negative (good) when wrong loss > correct loss
        #lambd = self.lambd if self.lambd else 1
        lambd = 0.2
        relative_score = correct_loss - lambd * (wrong_loss + correct_loss)
        loss = -relative_score

        return (loss, outputs) if return_outputs else loss


In [7]:
# main(args.model, args.middle_phrase, args.train_path, args.eval_path, args.contrastive, args.contrast_lambd, args.num_epochs, args.seed, learning_rate, args.cuda, args.dont_train, args.dont_eval, out_path, prefix_prompt=args.prefix, log_history=args.log_history, deepspeed=deepspeed, early_stopping=args.early_stopping)

#main("gpt-neo-sssm", "", "./data/lm_train_data/train.txt", "./data/lm_train_data/dev.txt", contrastive_train=False, contrastive_train_lambd=1, num_epochs=2, seed=42, lr=5e-5, use_cuda=True, dont_train=False, dont_eval=False, out_path=None, prefix_prompt=0, log_history=True, deepspeed=False, early_stopping=False)


In [8]:
main("flan-t5-base", "", "./data/lm_train_data/train.txt", "./data/lm_train_data/dev.txt", contrastive_train=False, contrastive_train_lambd=1, num_epochs=2, seed=42, lr=5e-5, use_cuda=True, dont_train=True, dont_eval=False, out_path=None, prefix_prompt=0, log_history=True, deepspeed=False, early_stopping=True)


[INFO|tokenization_utils_base.py:1852] 2023-10-12 16:39:56,258 >> loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/spiece.model
[INFO|tokenization_utils_base.py:1852] 2023-10-12 16:39:56,260 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/tokenizer.json
[INFO|tokenization_utils_base.py:1852] 2023-10-12 16:39:56,261 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1852] 2023-10-12 16:39:56,262 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/special_tokens_map.json
[INFO|tokenization_utils_base.py:1852] 2023-10-12 16:39:56,263 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/s

[INFO|configuration_utils.py:715] 2023-10-12 16:39:56,460 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/config.json
[INFO|configuration_utils.py:775] 2023-10-12 16:39:56,463 >> Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_s

[INFO|modeling_utils.py:2857] 2023-10-12 16:40:01,374 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/model.safetensors
[INFO|configuration_utils.py:768] 2023-10-12 16:40:01,434 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.33.0"
}

[INFO|modeling_utils.py:3643] 2023-10-12 16:40:05,994 >> All model checkpoint weights were used when initializing T5ForConditionalGeneration.

[INFO|modeling_utils.py:3651] 2023-10-12 16:40:05,995 >> All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


[INFO|configuration_utils.py:730] 2023-10-12 16:40:06,111 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/generation_config.json
[INFO|configuration_utils.py:768] 2023-10-12 16:40:06,112 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.33.0"
}

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
[INFO|language_modeling.py:130] 2023-10-12 16:40:12,861 >> Creating features from dataset file at ./data/lm_tra

[INFO|integration_utils.py:716] 2023-10-12 16:40:58,218 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Q: The girl had the flightiness of a sparrow: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a sparrow. The girl was very stable..' is more likely


Q: The girl had the flightiness of a rock: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a rock. The girl was very fickle..' is more likely


Q: It was as peaceful as a church.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a church.. It was very peaceful..' is more likely


Q: It was as peaceful as a battlefield.: 1. It was very peaceful. 2. It was full of conflict and danger, not peace.
model says 'It was as peaceful as a battlefield.. It was very peaceful..' is more likely


Q: The leaves were as green as grass: 1. The leaves were very green 2. The leaves were brown and not green at all.
model says 'The leaves were as green as grass. The leaves were very green.' is 

{'eval_loss': 0.5588154792785645,
 'accuracy (test)': 0.5457038391224863,
 'accuracy (dev)': 0.5457038391224863,
 'preds': [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0

In [ ]:
#3e-4

In [ ]:
#main("flan-t5-base", "", "./data/lm_train_data/train.txt", "./data/lm_train_data/dev.txt", contrastive_train=False, contrastive_train_lambd=1, num_epochs= 30, seed=42, lr=5e-5, use_cuda=True, dont_train=False, dont_eval=False, out_path=None, prefix_prompt=0, log_history=True, deepspeed=False, early_stopping=True, batch_size = 2)
main("flan-t5-base", "", "./data/lm_train_data/train.txt", "./data/lm_train_data/dev.txt", contrastive_train=False, contrastive_train_lambd=1, num_epochs= 30, seed=42, lr=1e-4, use_cuda=True, dont_train=False, dont_eval=False, out_path=None, prefix_prompt=0, log_history=True, deepspeed=False, early_stopping=True, batch_size = 4, patience_num = 4)


[INFO|tokenization_utils_base.py:1852] 2023-10-12 23:46:15,360 >> loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/spiece.model
[INFO|tokenization_utils_base.py:1852] 2023-10-12 23:46:15,361 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/tokenizer.json
[INFO|tokenization_utils_base.py:1852] 2023-10-12 23:46:15,362 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1852] 2023-10-12 23:46:15,363 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/special_tokens_map.json
[INFO|tokenization_utils_base.py:1852] 2023-10-12 23:46:15,364 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/s

Epoch,Training Loss,Validation Loss
1,No log,0.003890
2,No log,0.001824
3,0.006300,0.001708
4,0.006300,0.001828


[INFO|trainer.py:3115] 2023-10-12 23:47:49,640 >> ***** Running Evaluation *****
[INFO|trainer.py:3117] 2023-10-12 23:47:49,645 >>   Num examples = 1094
[INFO|trainer.py:3120] 2023-10-12 23:47:49,649 >>   Batch size = 8
[INFO|trainer.py:2841] 2023-10-12 23:48:09,546 >> Saving model checkpoint to ./lm_train_outputs/flan-t5-base_42/checkpoint-183
[INFO|configuration_utils.py:460] 2023-10-12 23:48:09,549 >> Configuration saved in ./lm_train_outputs/flan-t5-base_42/checkpoint-183/config.json
[INFO|configuration_utils.py:544] 2023-10-12 23:48:09,551 >> Configuration saved in ./lm_train_outputs/flan-t5-base_42/checkpoint-183/generation_config.json
[INFO|modeling_utils.py:1992] 2023-10-12 23:48:11,442 >> Model weights saved in ./lm_train_outputs/flan-t5-base_42/checkpoint-183/pytorch_model.bin
[INFO|trainer.py:2928] 2023-10-12 23:48:18,745 >> Deleting older checkpoint [lm_train_outputs/flan-t5-base_42/checkpoint-366] due to args.save_total_limit
/opt/conda/lib/python3.10/site-packages/torch/n